# Dynamic Workflows

To dynamically create an electron based on the output of another electron, use a sublattice.

## Context

A sublattice is a `lattice` wrapped in an `electron`, so all the restrictions of a lattice apply to it. Since it is also an electron, it is executed as a part of the workflow. Because of this dual identity, dynamic code can be run inside a sublattice which would otherwise be impossible in the Covalent framework. For example, sublattices provide a way to handle result-dependent loops and if/else statements.

## Best Practice

Use a sublattice (a `lattice` decorated with an `electron`) to encapsulate dynamic code that would otherwise be difficult or impossible to execute correctly in the Covalent paradigm. You can think of this as an extension to the technique of encapsulating execution-dependent decisions or loops in an electron. Using a sublattice enables you to compose and encapsulate arbitrarily complex code. Use the same best practices in building a sublattice that you would for any other lattice: [Do all calculations in electrons](./post_process.ipynb); [don't create unnecessary executors](./executor_assignment.ipynb); and so on.  


## Example

Contrast the two examples below.

### Example 1: Incorrect

The following example contains code in the `workflow_1` lattice that is not inside electrons: in this case, an if/else decision and a `for` loop. The workflow fails when Covalent tries to build the transport graph.

In [2]:
import covalent as ct

# Technique 1: Incorrect

@ct.electron
def task_1(x):
    return x * 2

@ct.electron
def task_2(x):
    return x ** 2

@ct.lattice
def workflow_1(a):
    
    res = task_1(a)
    
    # An if/else decision and a result-dependent loop with no enclosing electron
    if res < 10: 
        final_res = []
        for _ in range(res):
            final_res.append(task_2(res))
    else:
        final_res = res
    
    return final_res

# Uncomment these three lines to see the workflow fail
# id = ct.dispatch(workflow_1)(5)
# res = ct.get_result(id, wait=True)
# print(res)

### Example 2: Correct

The following code corrects the previous example by enclosing the if/else decision and the `for` loop in the `sub_workflow` sublattice. 

In [3]:
# Technique 2: Correct

# Define a sublattice that implements all the dynamic code
@ct.electron
@ct.lattice
def sub_workflow(res):
    
    if res < 10:
        final_res = []
        for _ in range(res):
            final_res.append(task_2(res))
    else:
        final_res = res
    
    return final_res


@ct.lattice
def workflow_2(a):
    res_1 = task_1(a)
    return sub_workflow(res_1) # Nothing to see here. Just an electron consuming the output of another electron.

id = ct.dispatch(workflow_2)(5)
res = ct.get_result(id, wait=True)
print(res)


Lattice Result
status: COMPLETED
result: 10
input args: ['5']
input kwargs: {}
error: None

start_time: 2023-03-14 19:41:25.136750
end_time: 2023-03-14 19:41:25.625968

results_dir: /Users/dave/.local/share/covalent/data
dispatch_id: 94509446-b719-42b1-b003-8aec0f1b9591

Node Outputs
------------
task_1(0): 10
:parameter:5(1): 5
:sublattice:sub_workflow(2): 10



## See Also

[Result-Dependent Loops](./result_dependent_loop.ipynb)

[Result-Dependent If-Else](./result_dependent_if_else.ipynb)